In [ ]:
import requests
import json 
import numpy as np

In [ ]:
def get_offer(address):

    headers = {
        'authority': 'www.xfinity.com',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'adrum': 'isAjax:true',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'referer': 'https://www.xfinity.com/planbuilder',
        'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    }

    with requests.Session() as s:

        # get a new set of cookies and a session id
        usercontext = s.get(
            'https://www.xfinity.com/sitecore/api/learn/neptune/persistencelayer/usercontext',
            #cookies=cookies,
            headers=headers,
        )

        # get the location id for the address of interests
        suggestions = s.get(
            f'https://www.xfinity.com/sitecore/api/learn/neptune/address-search/suggestions?show=false&search={address}',
            headers=headers,
        ).json()

        # now that we have cookies and the suggestions dict, we hit the localize API endpoint with the first suggestion to get the HK into the cookies
        json_data = {
            'AutoBuild': False,
            'AdditionalQueryParams': '',
            'Address': suggestions['Data'][0]['Address'],
            'RedirectURL': '/digital/offers/plan-builder',
            'RedirectionType': '',
            'OfferId': '',
            'OfferLobs': '',
            'XeppPromoId': '',
            'TenantInfo': {},
            'isACPOpted': False,
            'DisableRedirectUrl': False,
        }

        response = s.post(
            'https://www.xfinity.com/sitecore/api/learn/neptune/localize',
            headers=headers,
            json=json_data,
        )

        # and, at last, the offers API
        json_data = {
            'acpToggle': False,
        }

        response = s.post('https://digital.xfinity.com/offers/api/offerpackaging', headers=headers, json=json_data)

        return response.json()

In [ ]:
def process_offer(offer: json):
    # get inernet plans
    # intialize plans
    internet_plan_info = []
    # store content
    content =  offer['CONTENT']
    for plan_id in content.keys():
        plan = content[plan_id]
        # only keep offers with certain keys that indicate they are an internet plan
        if 'InternetPlanID' in plan.keys() and 'Name' in plan.keys():
            # store key information about each plan
            internet_plan_info.append([plan['InternetPlanID'], plan['DownloadSpeed'], plan['UploadSpeed'], plan['EdpPrice']])    

    # get cheapest plan
    min_price = min([plan[3] for plan in internet_plan_info])
    cheapest_plan = [plan for plan in internet_plan_info if plan[3] == min_price]

    # ensure if there is more than 1 plan, that they have hte same information
    if len(cheapest_plan)>1:
        if cheapest_plan[0] != cheapest_plan[1]:
            raise Warning('Multiple cheapest plans with different plan information.')

    # take first cheapest plan
    cheapest_plan = cheapest_plan[0]

    plan_name = cheapest_plan[0]
    price = cheapest_plan[3]

    # get download speed and units
    download_info = cheapest_plan[1].split(' ')
    speed_down = download_info[len(download_info)-2]
    speed_units = download_info[len(download_info)-1]

    # get upload speed and units
    upload_info = cheapest_plan[2].split(' ')
    speed_up = upload_info[len(upload_info)-2]

    return plan_name, price, speed_down, speed_up, speed_units

In [ ]:
address = '1274 beacon st, waban'
offer1 = get_offer(address)
process_offer(offer1)